In [1]:
import os, sys, re
from functools import partial
from collections import defaultdict, Counter

import pandas as pd
import numpy as np

from bibtexparser.latexenc import string_to_latex

sys.path.insert(1, os.path.join(sys.path[0], '..'))

import database
from snowballing.operations import load_work_map_all_years, work_to_bibtex, reload
from snowballing.operations import match_bibtex_to_work
from snowballing.approaches import get_approaches, name, wlatex_name, wcitea
reload()

all_approaches = get_approaches()
script = [(a, m) for a, m in all_approaches if not m["binary"]]
binary = [(a, m) for a, m in all_approaches if m["binary"]]

approaches = [a for a, m in script]

len(script)

27

In [2]:
with open('../../csur/bibliography.bib') as bibtex_file:
    bibtex_str = bibtex_file.read()

matched = match_bibtex_to_work(bibtex_str.split("%Entries")[-1])
works = dict(map(reversed, matched))
latex_name = partial(wlatex_name, works=works)
citea = partial(wcitea, works=works)

In [3]:
languages = defaultdict(list)

for a, m in script:
    for lang in m['languages']:
        languages[repr(lang)].append((a, m))
languages["Agnostic"] = languages["*"]
del languages["*"]
del languages["C"]
del languages["C++"]
del languages["Java"]

count = Counter({k: len(v) for k, v in languages.items()})

In [4]:
count.most_common()

[('Python', 16),
 ('R', 6),
 ('Agnostic', 3),
 ('S', 1),
 ('IDL', 1),
 ('Perl', 1),
 ('Blockly', 1),
 ('LLVM', 1),
 ('Matlab', 1)]

In [5]:
columns = ["Language", "Count", "Approaches"]
data = [
    [language, str(number), ", ".join(latex_name(a) for a, m in languages[language])]
    for language, number in count.most_common()
]

In [9]:
header = r"""\begin{tabular}{|c|c|m{0.7\textwidth}|}
  \hline
  \rowcolor{BlueRow}
  %s\\
  \hline
""" % (
    " & ".join([
        (r"\multicolumn{1}{%sc|}{\textbf{%s}}" % ('|' if not i else '', column,))
        for i, column in enumerate(columns)]),
)
footer = r"""
  \hline
\end{tabular}
"""


In [10]:
result = []
for i, row in enumerate(data):
    if i % 2 == 1:
        result.append(r"  \rowcolor{BlueRow}")
    result.append("  " + " & ".join(row) + r"\\")
    #result.append(r"  \hline")

In [11]:
latex = header + "\n" + "\n".join(result) + "\n" + footer
with open("../../csur/figs/script_languages.tex", "w") as tex:
    tex.write(latex)